<a href="https://colab.research.google.com/github/beny2000/HeartDiseaseClassifier/blob/main/Final_KNN_B%26E.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#imports
import sys
import pandas as pd
import numpy as np
import sklearn
import matplotlib
import keras
import math
from sklearn import preprocessing
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import scale
import pickle

!pip install anvil-uplink
import anvil.server
anvil.server.connect("2ZZFEIO7RSWJ2NSXVOIJU7NK-PUMVCYIB7OL5ZPSL")



Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default environment (dev)" as SERVER


KNN


In [2]:
#read data
emily = pd.read_csv('heart.csv')
data=emily

#split into features and labels
X = data.iloc[:,:-1].values
y = data.iloc[:,13].values

#split into test & train
x_train, x_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

#preprocess data
scaler = RobustScaler().fit(X)
x_Train = scaler.transform(x_train)
x_Test = scaler.transform(x_test)

#build model
model = KNeighborsClassifier(n_neighbors=5)
model.fit(x_Train, y_train)

#save model
pickle.dump(model, open('model_new.pickle', 'wb'))

In [3]:
#Check model performance on the test set
print('Accuracy of K-NN classifier on training set: {:.2f}'
     .format(model.score(x_Train, y_train)))
print('Accuracy of K-NN classifier on test set: {:.2f}'
     .format(model.score(x_Test, y_test)))
print("Pediction for", x_Test[0], model.predict([x_Test[0]]), "Correct prediction:", y_test[0])

Accuracy of K-NN classifier on training set: 0.89
Accuracy of K-NN classifier on test set: 0.82
Pediction for [ 0.44444444 -1.         -0.5         0.75        1.05511811  0.
 -1.         -0.21538462  1.          0.125       0.          0.
  1.        ] [0] Correct prediction: 0


In [4]:
#Make predict using model

#[age, sex, cp, trestbps, chol, fbs, restecg, thalach, exang, oldpeak, slope, ca, thal]
#input = [51,	0,	0,	130,	305,	0,	1,	142,	1,	1.2,	1,	0,	3] #0
#input = [63,	1,	3,	145,	233,	1,	0,	150,	0,	2.3,	0,	0,	1] #1

@anvil.server.callable
def knnpredict(input):
  #loads model and makes prediction on input data

  model = pickle.load(open('model_new.pickle', 'rb'))
  input_scaled = scaler.transform([np.array(input)])

  return model.predict(input_scaled)[0], model.score(x_Test, y_test)


In [ ]:
anvil.server.wait_forever()